In [5]:
## This chunk of code was given my Mathilde via Slack 
## on Thurs, Jun 20 @ 8:55 am

# Mathilde suggests to integrate this in a script that produces 
# xdmf/h5 files to look at the result on supermuc.

# You need as input file either the fault output from seisol 
# (you need only the first time step and Ts0, Td0, and Pn0, mu_d) 
# or the output file of read_ini_fault_parameter.py (with the same variables)

# what would be convenient would be to modify  read_ini_fault_parameter.py 
# to add the part that compute R, so everything is done in one step

import seissolxdmf
import seissolxdmfwriter as sxw
import numpy as np
import glob
import os


jobid = '3457692'

def find_xdmffile_wahweap(jobid):
    """Find XDMF file on wahweap. 
    Given a jobid, returns a string which is the full pat to the extracted-fault.xdmf file on Wahweap laptop

    Args:
        jobid (string): i.e. '3438438'
    Return: 
        xdmfFilename (string): full path to the extracted-fault.xdmf file matching the jobid
    """
    # Example full path to xdmf:
    # /Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/data_tmp/dynamic-rupture/FL33-only/jobid_3440215/output_jobid_3440215_extracted-fault.xdmf

    datadir = '/Users/hyin/ags_local/data/haiti_seissol_data/'
    xdmfFilename = glob.glob(datadir + 'dynamic-rupture/*/jobid_'+jobid+'/*extracted-fault.xdmf')[0]
    return xdmfFilename

xdmfFilename = find_xdmffile_wahweap(jobid)

In [6]:
sx = seissolxdmf.seissolxdmf(xdmfFilename)

# x, y, z components of each point where the surface displacement is defined. 
xyz = sx.ReadGeometry() # xyz.shape is (16347, 3) np arrray
connect = sx.ReadConnect()  # connect.shape is (32526, 3) np array


# define the barycenter of each element
barycenter = (
    xyz[connect[:, 0], :]
    + xyz[connect[:, 1], :]
    + xyz[connect[:, 2], :]
) / 3.0

# define cohesion in each element
cohesion = np.maximum((barycenter[:, 2] + 3000) / 3000, 0) * 1e6 + 0.5e6

# define time step you wish to extract (@todo: is this right?)
dt = sx.ReadTimeStep()
outputTimes = sx.ReadTimes()
idt = int(outputTimes[0])    # Pull first time step
idt

# Pull data from the xdmf file
mu_st = sx.ReadData("Mud", idt) # @todo: this variable is not output in output-fault.xdmf
Ts0 = sx.ReadData("Ts0", idt)
Td0 = sx.ReadData("Td0", idt)
Pn0 = abs(sx.ReadData("Pn0", idt))


# manually define dynamic coefficient of friction
mu_dy = 0.2

# Calculate the value of R. @todo: is there a good cite for this formula? 
myData = (np.sqrt(np.power(Ts0, 2) + np.power(Td0, 2)) - mu_dy * Pn0) / (
    (mu_st - mu_dy)* Pn0 + cohesion
)

## Rewrite this data to a new xdmf file

In [7]:

# Pull other data fields
geom = sx.ReadGeometry()
connect = sx.ReadConnect()

# Write all selected time steps to files dCFS-fault.xdmf/dCFS-fault.h5
outpath = os.path.dirname(xdmfFilename)
outfile = outpath + '/jobid_'+jobid+'_R-fault'
sxw.write(
    outfile,
    geom,
    connect,
    {"R": myData},
    {idt: 0},
    reduce_precision=True,
    backend="hdf5",
)

done writing /Users/hyin/ags_local/data/haiti_seissol_data/dynamic-rupture/FL33-only/jobid_3457692/jobid_3457692_R-fault.xdmf
full path: /Users/hyin/ags_local/data/haiti_seissol_data/dynamic-rupture/FL33-only/jobid_3457692/jobid_3457692_R-fault.xdmf
done writing /Users/hyin/ags_local/data/haiti_seissol_data/dynamic-rupture/FL33-only/jobid_3457692/jobid_3457692_R-fault.h5
